In [11]:
import torch
'''
torch.max 函数的demo
num = 10
a = torch.arange(num).reshape(num,1)
b = 5*torch.ones(num,1)
c = torch.cat((a-b, torch.zeros(num,1)), 1)
d = torch.max(c, 1)
print(d.values.reshape(10, 1))
'''

'\ntorch.max 函数的demo\nnum = 10\na = torch.arange(num).reshape(num,1)\nb = 5*torch.ones(num,1)\nc = torch.cat((a-b, torch.zeros(num,1)), 1)\nd = torch.max(c, 1)\nprint(d.values.reshape(10, 1))\n'

需要求解的问题是：
$$\frac{\partial u}{\partial t}+\frac{1}{2} \sigma^2 S^2 \frac{\partial^2 u}{\partial S^2}+r S \frac{\partial u}{\partial S}-r u=0$$
$$u(0, t)=0 , u\left(S_{\max }, t\right)=S_{\max }-K,$$
$$
u(S, T)=\max (S-K, 0),
$$
其中$S$是原生资产价格，$t$是时间，$r$是无风险利率，$\sigma$是波动率.
$$sigma = 0.3;\quad          \% volatility$$
$$r = 0.25;\quad              \% interest rate $$
$$K = 10; \quad               \% strike price$$
$$S_{\max } = 50; \quad               \% asset value$$
$$T = 1; \quad                \% time$$

In [12]:
# Domain and Sampling
def interior(n=1000):
    S = 50*torch.rand(n, 1)
    t = torch.rand(n, 1)
    # 0
    cond = torch.zeros_like(S)
    return S.requires_grad_(True), t.requires_grad_(True), cond


def up(n=100):
    S = 50*torch.rand(n, 1)
    t = torch.ones_like(S)
    # max(S-K,0)
    cond = torch.max(torch.cat(((S-10*torch.ones(n,1)), torch.zeros(n,1)),1), 1)
    return S.requires_grad_(True), t.requires_grad_(True), cond


def left(n=100):
    t = torch.rand(n, 1)
    S = torch.zeros_like(t)
    # 0
    cond = torch.zeros_like(S)
    return S.requires_grad_(True), t.requires_grad_(True), cond


def right(n=100):
    t = torch.rand(n, 1)
    S = 50*torch.ones_like(t)
    # S_max-K
    cond = S - 10*torch.ones_like(S)
    return S.requires_grad_(True), t.requires_grad_(True), cond

定义$2\times 32\times 32\times 32\times 1$的神经网络，激活函数用Tanh

In [13]:
# Neural Network
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(2, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)

损失函数：

In [14]:
# Loss
loss = torch.nn.MSELoss()


def gradients(u, x, order=1):
    if order == 1:
        return torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u),
                                   create_graph=True,
                                   only_inputs=True, )[0]
    else:
        return gradients(gradients(u, x), x, order=order - 1)


    # x -> S, y -> t
def l_interior(u):
    x, y, cond = interior()
    uxy = u(torch.cat([x, y], dim=1))
    return loss(gradients(uxy, y, 1) \
                + 0.5*0.09*x**2*gradients(uxy, x, 2) \
                + 0.25*x*gradients(uxy, x, 1) \
                - 0.25*uxy\
                , cond)


def l_up(u):
    x, y, cond = up()
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)


def l_left(u):
    x, y, cond = left()
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)


def l_right(u):
    x, y, cond = right()
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)


In [15]:
# Training
u = MLP()
opt = torch.optim.Adam(params=u.parameters())
for i in range(10000):
    opt.zero_grad()
    l = l_interior(u) \
        + l_up(u) \
        + l_left(u) \
        + l_right(u)
    l.backward()
    opt.step()
    if (i + 1) % 1000 == 0:
        print("{:.2%}".format((i + 1)/10000))

AttributeError: 'torch.return_types.max' object has no attribute 'size'

In [ ]:
# Inference
xc = torch.linspace(0, 1, 100)
xx, yy = torch.meshgrid(50*xc, xc)
xx = xx.reshape(-1, 1)
yy = yy.reshape(-1, 1)
xy = torch.cat([xx, yy], dim=1)
u_pred = u(xy)